#General imports

In [28]:
# Used imports -- standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# miscelaneous imports
import gc


#Reading csv into dataframe

In [29]:
df_deelnemeners = pd.read_csv('Data - Zuivere dataset/HealthyLIFE_datadump_03-2023_1.csv', sep=';')
df_fitheids_scan = pd.read_csv('Data - Zuivere dataset/HealthyLIFE_datadump_03-2023_2.csv', sep=';')
df_cbs_afstanden = pd.read_csv('Data - Zuivere dataset/Nabijheid_voorzieningen__regio_17052023_115259.csv', sep=';')
df_kvk_inkomen = pd.read_csv('Data - Zuivere dataset/kvk2019-wb2022-inkomen-1.csv', sep=';')
df_afkomst_per_gemeente = pd.read_csv('Data - Zuivere dataset/Afkomst_Per_Gemeente.csv',sep=";" )
df_uitkeringen = pd.read_csv('Data - Zuivere dataset/Uitkeringen.csv', sep=';')
df_welzijn = pd.read_csv('Data - Zuivere dataset/table_tabel-34911035-1268-41a0-961d-229504cfd3f2.csv')
df_gezondheid = pd.read_csv('Data - Zuivere dataset/Gezondheidsmonitor__bevolking_18_jaar_of_ouder__regio__2020_17052023_132409.csv', sep=';')

In [30]:
df_deelnemeners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426 entries, 0 to 2425
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id               2426 non-null   int64 
 1   Geslacht         2426 non-null   object
 2   Woonplaats       2426 non-null   object
 3   Postcode         1702 non-null   object
 4   Beweegcoach1     1609 non-null   object
 5   Leefstijlcoach1  2214 non-null   object
 6   age              2426 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 132.8+ KB


In [31]:
df_deelnemeners.rename(columns={'Id': 'DeelnemerID'}, inplace=True)

# Adding columns
df_deelnemeners['Gemeente'] = df_deelnemeners['Woonplaats']

# Replace values
df_deelnemeners['Gemeente'] = df_deelnemeners['Gemeente'].replace(['Born', 'Geleen', 'Sittard'], 'Sittard-Geleen')
df_deelnemeners['Gemeente'] = df_deelnemeners['Gemeente'].replace(['Schinnen'], 'Beekdaelen')

# Dropping row with specific testing value
df_deelnemeners.drop(df_deelnemeners[df_deelnemeners['Gemeente'] == 'Test'].index, inplace = True)

# Dropping columns
df_deelnemeners.drop(columns=['Woonplaats', 'Postcode'], inplace = True)

# Dit is een beetje experimenteel maar hier drop ik gewoon waardeloze data die we niet nodig hebben -groetjes Justin XOXOXO
df_deelnemeners.drop(columns={'Beweegcoach1', 'Leefstijlcoach1'}, inplace= True)
# Ook doe ik even hier de deelnemerID naar int omzetten omdat er geen halve deelnemen kan zijn... denk ik
df_deelnemeners['DeelnemerID'] = df_deelnemeners['DeelnemerID'].astype(int)

# Display the modified DataFrame
df_deelnemeners

,DeelnemerID,Geslacht,age,Gemeente
0,1,V,51,Sittard-Geleen
1,2,V,45,Sittard-Geleen
2,3,V,68,Sittard-Geleen
3,4,M,64,Sittard-Geleen
4,5,V,63,Sittard-Geleen
...,...,...,...,...
2421,2661,V,47,Beekdaelen
2422,2662,V,29,Sittard-Geleen
2423,2663,V,66,Maasgouw
2424,2664,M,52,Maasgouw


In [32]:
df_fitheids_scan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DeelnemerID  3193 non-null   float64
 1   Datum        3197 non-null   object 
 2   Lengte       3195 non-null   object 
 3   Gewicht      3178 non-null   object 
 4   6 MWT        3120 non-null   object 
 5   TUG          3130 non-null   object 
 6   BMI          3196 non-null   object 
 7   Conditie     3197 non-null   object 
 8   Lenigheid    3006 non-null   object 
 9   Knijpkracht  3197 non-null   object 
 10  #meting      3197 non-null   int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 274.9+ KB


In [33]:
# Convert 'DeelnemerID' column to int
df_fitheids_scan['DeelnemerID'] = df_fitheids_scan['DeelnemerID'].astype(pd.Int64Dtype())

# Replace commas with periods and convert columns to float
columns_to_convert = ['Lengte', 'Gewicht', '6 MWT', 'TUG', 'BMI', 'Conditie', 'Lenigheid', 'Knijpkracht']
df_fitheids_scan[columns_to_convert] = df_fitheids_scan[
    columns_to_convert].astype(str).apply(lambda x: x.str.replace(',', '.')).astype(float)

df_fitheids_scan['Datum'] = pd.to_datetime(df_fitheids_scan['Datum'], format='%d/%m/%Y').dt.normalize()
df_fitheids_scan['Datum'] = df_fitheids_scan['Datum'].dt.strftime('%Y-%m-%d')

# Deleting unused dataframes
del columns_to_convert

# Display the modified DataFrame
df_fitheids_scan

,DeelnemerID,Datum,Lengte,Gewicht,6 MWT,TUG,BMI,Conditie,Lenigheid,Knijpkracht,#meting
0,1,2018-03-30,159.0,97.6,500.0,6.36,38.606068,0.820291,31.50,15.0,1
1,1,2018-06-18,159.0,95.3,448.0,9.30,37.700000,0.695091,0.00,45.8,2
2,2,2018-03-30,175.0,89.1,596.0,5.82,29.093878,0.806495,27.00,34.0,1
3,2,2019-01-25,170.0,86.0,556.0,5.50,29.757785,0.779192,29.25,29.0,3
4,3,2018-07-02,151.5,94.8,512.0,6.84,41.303140,1.054387,19.00,20.3,2
...,...,...,...,...,...,...,...,...,...,...,...
3192,2409,2023-03-07,182.0,130.0,543.5,6.00,39.246468,0.834113,NaN,45.6,1
3193,2341,2023-03-07,184.5,139.2,558.0,6.20,40.892767,0.766821,14.50,84.7,2
3194,2340,2023-03-07,158.6,107.4,486.0,7.00,42.697054,0.788393,486.00,33.7,2
3195,2337,2023-03-07,175.5,111.0,557.0,6.40,36.038669,0.820348,25.50,68.2,2


In [34]:
df_fitheids_scan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DeelnemerID  3193 non-null   Int64  
 1   Datum        3197 non-null   object 
 2   Lengte       3195 non-null   float64
 3   Gewicht      3178 non-null   float64
 4   6 MWT        3120 non-null   float64
 5   TUG          3130 non-null   float64
 6   BMI          3196 non-null   float64
 7   Conditie     3197 non-null   float64
 8   Lenigheid    3006 non-null   float64
 9   Knijpkracht  3197 non-null   float64
 10  #meting      3197 non-null   int64  
dtypes: Int64(1), float64(8), int64(1), object(1)
memory usage: 278.0+ KB


In [35]:
df_cbs_afstanden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Columns: 127 entries, Regio's to Afstand tot brandweerkazerne (km)
dtypes: float64(92), int64(1), object(34)
memory usage: 1.9+ MB


In [36]:
df_cbs_afstanden

,Regio's,Perioden,Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpraktijk (km),Gezondheid en welzijn/Huisartsenpraktijk/Aantal huisartsenpraktijken/Binnen 1 km (aantal),Gezondheid en welzijn/Huisartsenpraktijk/Aantal huisartsenpraktijken/Binnen 3 km (aantal),Gezondheid en welzijn/Huisartsenpraktijk/Aantal huisartsenpraktijken/Binnen 5 km (aantal),Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpost (km),Gezondheid en welzijn/Afstand tot apotheek (km),Gezondheid en welzijn/Ziekenhuis (incl. buitenpolikliniek)/Afstand tot ziekenhuis (km),Gezondheid en welzijn/Ziekenhuis (incl. buitenpolikliniek)/Aantal ziekenhuizen/Binnen 5 km (aantal),...,Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 5 km (aantal),Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 10 km (aantal),Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal),Vrije tijd en cultuur/Recreatie/Afstand tot sauna (km),Vrije tijd en cultuur/Recreatie/Afstand tot zonnebank (km),Vrije tijd en cultuur/Recreatie/Afstand tot attractie (km),Vrije tijd en cultuur/Recreatie/Aantal attracties/Binnen 10 km (aantal),Vrije tijd en cultuur/Recreatie/Aantal attracties/Binnen 20 km (aantal),Vrije tijd en cultuur/Recreatie/Aantal attracties/Binnen 50 km (aantal),Afstand tot brandweerkazerne (km)
0,Nederland,2018,1.0,1.7,9.5,20.2,6.3,1.2,4.7,1.2,...,1.3,3.0,7.4,8.1,4.1,6.1,2.4,7.1,32.9,2.1
1,Nederland,2019,1.0,1.7,9.6,20.2,6.3,1.2,4.8,1.2,...,1.3,2.9,7.4,8.1,4.1,5.8,2.7,7.9,35.9,2.1
2,Nederland,2020,1.0,1.5,8.5,18.0,6.4,1.2,4.8,1.2,...,1.3,2.9,7.4,8.0,4.1,5.7,2.9,8.3,37.5,2.1
3,Nederland,2021,1.0,1.5,8.3,17.6,6.5,1.2,4.8,1.2,...,1.3,2.9,7.2,7.6,4.0,5.2,3.4,9.9,44.7,2.1
4,Aa en Hunze,2018,2.5,0.5,1.2,2.3,11.2,2.5,10.7,0.0,...,0.0,0.3,1.8,9.6,10.5,6.0,1.9,8.1,25.6,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,Zwijndrecht,2021,0.7,1.4,6.4,17.6,6.5,0.9,2.5,0.9,...,0.4,2.8,10.2,5.2,1.6,3.6,1.2,12.7,67.3,1.7
1984,Zwolle,2018,0.9,1.5,7.8,15.7,4.5,1.2,4.5,0.6,...,0.8,1.0,3.0,8.7,1.6,3.6,2.6,3.8,27.6,2.6
1985,Zwolle,2019,1.0,1.4,7.7,15.5,4.2,1.2,4.2,0.6,...,0.8,1.0,3.0,8.4,1.7,3.6,2.6,4.5,29.7,2.6
1986,Zwolle,2020,0.9,1.3,7.2,14.6,4.2,1.2,4.2,0.6,...,0.8,1.0,3.0,8.4,1.7,3.5,2.6,4.5,33.4,2.6


In [37]:
df_cbs_afstanden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Columns: 127 entries, Regio's to Afstand tot brandweerkazerne (km)
dtypes: float64(92), int64(1), object(34)
memory usage: 1.9+ MB


In [38]:
# rename columns for merging
df_cbs_afstanden.rename(columns={"Regio's": 'Gemeente'}, inplace=True)

# Replace the exact string '       .' with NaN values
df_cbs_afstanden.replace('       .', np.nan, inplace=True)

# Drop the columns containing NaN values
df_cbs_afstanden.dropna(axis=1, how="all", inplace=True)

# Drop all rows containing the word aantal
df_cbs_afstanden = df_cbs_afstanden.filter(regex=r'^(?!.*aantal).*$', axis=1)

# Because there a some municipalities with many nan values, we drop them because the clog up the data
df_cbs_afstanden.dropna(inplace=True)

# Display the modified DataFrame
df_cbs_afstanden

C:\Users\justi\AppData\Local\Temp\ipykernel_3148\665933816.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cbs_afstanden.dropna(inplace=True)


,Gemeente,Perioden,Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpraktijk (km),Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpost (km),Gezondheid en welzijn/Afstand tot apotheek (km),Gezondheid en welzijn/Ziekenhuis (incl. buitenpolikliniek)/Afstand tot ziekenhuis (km),Gezondheid en welzijn/Ziekenhuis (excl. buitenpolikliniek)/Afstand tot ziekenhuis (km),Detailhandel/Winkels dagelijkse boodschappen/Afstand tot grote supermarkt (km),Detailhandel/Winkels dagelijkse boodschappen/Afstand tot ov. dagel. levensmiddelen (km),Detailhandel/Warenhuis/Afstand tot warenhuis (km),...,Vrije tijd en cultuur/Sport/Afstand tot zwembad (km),Vrije tijd en cultuur/Sport/Afstand tot kunstijsbaan (km),Vrije tijd en cultuur/Museum/Afstand tot museum (km),Vrije tijd en cultuur/Podiumkunsten (excl. festivals)/Afstand tot podiumkunsten totaal (km),Vrije tijd en cultuur/Podiumkunsten (excl. festivals)/Afstand tot poppodium (km),Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km),Vrije tijd en cultuur/Recreatie/Afstand tot sauna (km),Vrije tijd en cultuur/Recreatie/Afstand tot zonnebank (km),Vrije tijd en cultuur/Recreatie/Afstand tot attractie (km),Afstand tot brandweerkazerne (km)
0,Nederland,2018,1.0,6.3,1.2,4.7,7.0,0.9,0.8,2.7,...,3.3,18.6,3.9,4.9,11.0,6.3,8.1,4.1,6.1,2.1
1,Nederland,2019,1.0,6.3,1.2,4.8,7.1,0.9,0.8,2.6,...,3.2,18.5,3.9,4.8,10.9,6.3,8.1,4.1,5.8,2.1
2,Nederland,2020,1.0,6.4,1.2,4.8,7.1,0.9,0.8,2.6,...,3.2,18.5,4.0,4.8,10.2,6.4,8.0,4.1,5.7,2.1
3,Nederland,2021,1.0,6.5,1.2,4.8,7.1,0.9,0.8,2.6,...,3.2,18.5,4.1,4.8,10.1,6.5,7.6,4.0,5.2,2.1
4,Aa en Hunze,2018,2.5,11.2,2.5,10.7,11.5,2.1,1.9,5.2,...,8.1,32.2,7.9,11.3,28.7,11.6,9.6,10.5,6.0,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,Zwijndrecht,2021,0.7,6.5,0.9,2.5,2.5,0.7,0.6,2.6,...,2.2,6.9,4.9,4.1,6.3,5.7,5.2,1.6,3.6,1.7
1984,Zwolle,2018,0.9,4.5,1.2,4.5,4.5,0.8,0.9,1.6,...,6.5,30.8,8.5,3.0,3.5,3.5,8.7,1.6,3.6,2.6
1985,Zwolle,2019,1.0,4.2,1.2,4.2,4.2,0.9,0.9,1.5,...,6.4,30.4,8.4,2.9,3.4,3.4,8.4,1.7,3.6,2.6
1986,Zwolle,2020,0.9,4.2,1.2,4.2,4.2,0.8,0.9,1.5,...,6.4,30.5,8.4,2.9,3.4,3.4,8.4,1.7,3.5,2.6


In [39]:
df_cbs_afstanden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446 entries, 0 to 1987
Data columns (total 34 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   Gemeente                                                                                     1446 non-null   object 
 1   Perioden                                                                                     1446 non-null   int64  
 2   Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpraktijk (km)                 1446 non-null   float64
 3   Gezondheid en welzijn/Huisartsenpraktijk/Afstand tot huisartsenpost (km)                     1446 non-null   float64
 4   Gezondheid en welzijn/Afstand tot apotheek (km)                                              1446 non-null   float64
 5   Gezondheid en welzijn/Ziekenhuis (incl.

In [40]:
# Filter and sort the rows for the years ranging from 2018 to 2021
df_chunks = {}
years = range(2018, 2022)
for year in years:
    df_year = df_cbs_afstanden[df_cbs_afstanden['Perioden'] == year]
    df_chunks[year] = df_year

# Access individual DataFrames for each year
df_2018 = df_chunks[2018]
df_2019 = df_chunks[2019]
df_2020 = df_chunks[2020]
df_2021 = df_chunks[2021]

# Deleting redundancies
del df_chunks, years, year, df_year

In [41]:
# NL -- Hier is een probleem dat de data op het CSV niveau geprepareerd is. Dit is niet de bedoeling omdat we nu heel veel gemeentes missen.
df_gezondheid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 21 columns):
 #   Column                                                                         Non-Null Count  Dtype 
---  ------                                                                         --------------  ----- 
 0   Gemeente                                                                       12 non-null     object
 1   Ervaren gezondheid (goed/zeer goed) (%)                                        12 non-null     object
 2   Eén of meer langdurige aandoeningen  (%)                                       12 non-null     object
 3   Functiebeperkingen|Eén of meer lichamelijke beperkingen  (%)                   12 non-null     object
 4   Functiebeperkingen|Soort beperking|Beperking in horen  (%)                     12 non-null     object
 5   Functiebeperkingen|Soort beperking|Beperking in zien (%)                       12 non-null     object
 6   Functiebeperkingen|Soort beperking|B

In [42]:
df_kvk_inkomen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3684 entries, 0 to 3683
Data columns (total 6 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   Gemeentecode                                             345 non-null    object
 1   Wijkcode                                                 3683 non-null   object
 2   Regionaam                                                3684 non-null   object
 3   Gemiddeld persoonlijk inkomen per inwoner (x 1000 euro)  3684 non-null   object
 4   40 procent huishoudens met laagste besteedbaar inkomen   3684 non-null   object
 5   20 procent huishoudens met hoogste besteedbaar inkomen   3684 non-null   object
dtypes: object(6)
memory usage: 172.8+ KB


In [43]:
# drops all rows that contain nan values in the column 'Gemeentenaam', this is every row except the total values
df_kvk_inkomen.dropna(subset=['Gemeentecode'], inplace=True)

# Drop column Gemeentecode, Wijkcode, 40 procent huishoudens met laagste besteedbaar inkomen, 20 procent huishoudens met hoogste besteedbaar inkomen
# This is done to only get the data for the median income per person in the municipality
df_kvk_inkomen.drop(columns=['Gemeentecode', 'Wijkcode', '40 procent huishoudens met laagste besteedbaar inkomen', '20 procent huishoudens met hoogste besteedbaar inkomen'], inplace=True)

# Remove commas and convert the column to float
df_kvk_inkomen['Gemiddeld persoonlijk inkomen per inwoner (x 1000 euro)'] = df_kvk_inkomen['Gemiddeld persoonlijk inkomen per inwoner (x 1000 euro)'].str.replace(',', '.').astype(float)

# Reset the index for readability purposes
df_kvk_inkomen.reset_index(inplace=True, drop=True)

# renames the column 'Gemiddeld persoonlijk inkomen per inwoner (x 1000 euro)' to 'gemiddeld inkomen'
df_kvk_inkomen.rename(columns={'Gemiddeld persoonlijk inkomen per inwoner (x 1000 euro)': 'gemiddeld inkomen'}, inplace=True)

# Multiply the column 'gemiddeld inkomen' by 1000 to get the actual value
df_kvk_inkomen['gemiddeld inkomen'] = df_kvk_inkomen['gemiddeld inkomen'] * 1000

# Rename columns Regionaam to Gemeente to match the other dataframes
df_kvk_inkomen.rename(columns={'Regionaam': 'Gemeente'}, inplace=True)


In [44]:
df_kvk_inkomen

,Gemeente,gemiddeld inkomen
0,Totaal_Nederland,27100.0
1,Almere,26200.0
2,Stadskanaal,21800.0
3,Veendam,22900.0
4,Zeewolde,27700.0
...,...,...
340,Molenlanden,26400.0
341,Eemsdelta,23500.0
342,Dijk en Waard,26400.0
343,Land van Cuijk,26200.0


In [45]:
# Calculate the median income per municipality and display it
median_kvk_inkomen_Nederland_precies = df_kvk_inkomen['gemiddeld inkomen'].median()
mean_kvk_inkomen_Nederland_precies = df_kvk_inkomen['gemiddeld inkomen'].mean()

median_kvk_inkomen_Nederland_afgerond = round(median_kvk_inkomen_Nederland_precies)
mean_kvk_inkomen_Nederland_afgerond = round(mean_kvk_inkomen_Nederland_precies)

print(f'De mediaan voor alle inkomens is: {median_kvk_inkomen_Nederland_afgerond}.\nHet gemiddelde voor alle inkomens is: {mean_kvk_inkomen_Nederland_afgerond}.')

del mean_kvk_inkomen_Nederland_precies, mean_kvk_inkomen_Nederland_afgerond, median_kvk_inkomen_Nederland_afgerond, median_kvk_inkomen_Nederland_precies

De mediaan voor alle inkomens is: 26500.
Het gemiddelde voor alle inkomens is: 27142.


In [46]:
# Groot probleem met deze dataset is overigens dat het wederom niet alle gemeenten bevat. Dit is uiters vervelend en moet opgelost worden. :(
# Deze code is overigens modulaire en kan dus makkelijk aangepast worden voor andere jaren. :)

# Rename columns
df_afkomst_per_gemeente.rename(columns={'Gemeentenaam': 'Gemeente'}, inplace=True)

# Making a new dataframe that only contains all non eropean immigrants

# Create a list of columns to be combined into 'Other (%)'
unused_columns = ['Marokko (%)', 'Nederlandse Antillen en Aruba (%)', 'Suriname (%)', 'Turkije (%)', 'Overig niet-westers (%)']

# Create the 'Other (%)' column and initialize it with zeros
df_afkomst_per_gemeente['Other (%)'] = 0

# Sum up the values from unused columns into 'Other (%)'
df_afkomst_per_gemeente['Other (%)'] = df_afkomst_per_gemeente[unused_columns].sum(axis=1)

# Keep only the desired columns
desired_columns = ['Gemeente', 'Autochtoon (%)', 'Westers totaal (%)', 'Other (%)']
df_afkomst_per_gemeente_gegeneraliseerd = df_afkomst_per_gemeente[desired_columns]

del unused_columns, desired_columns

In [47]:
# Display the normal DataFrame
df_afkomst_per_gemeente

,Gemeente,Autochtoon (%),Westers totaal (%),Marokko (%),Nederlandse Antillen en Aruba (%),Suriname (%),Turkije (%),Overig niet-westers (%),Other (%)
0,Heerlen,70,18,3,2,0,1,6,12
1,Weert,77,12,4,1,0,1,4,10
2,Beekdaelen,84,12,0,0,0,0,3,3
3,Breda,75,12,3,1,1,2,6,13
4,Echt-Susteren,82,13,0,0,0,0,4,4
5,Kerkrade,68,25,1,1,0,1,4,7
6,Maasgouw,85,12,0,0,0,0,3,3
7,Roerdalen,82,14,0,0,0,0,3,3
8,Roermond,69,16,3,1,1,4,7,16
9,Sittard-Geleen,77,15,2,0,0,1,5,8


In [48]:
# Display the modified DataFrame
df_afkomst_per_gemeente_gegeneraliseerd

,Gemeente,Autochtoon (%),Westers totaal (%),Other (%)
0,Heerlen,70,18,12
1,Weert,77,12,10
2,Beekdaelen,84,12,3
3,Breda,75,12,13
4,Echt-Susteren,82,13,4
5,Kerkrade,68,25,7
6,Maasgouw,85,12,3
7,Roerdalen,82,14,3
8,Roermond,69,16,16
9,Sittard-Geleen,77,15,8


In [49]:
df_uitkeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 14 columns):
 #   Column                                                                                                              Non-Null Count  Dtype 
---  ------                                                                                                              --------------  ----- 
 0   Perioden                                                                                                            396 non-null    object
 1   Plaats                                                                                                              396 non-null    object
 2   Uitkeringsontvangers, totaal |Uitkeringsontvangers, totaal                                                          396 non-null    int64 
 3   Uitkeringsontvangers, totaal |Tot de AOW- leeftijd                                                                  396 non-null    int64 
 4   Uitkeringsontvangers, tota

In [50]:
# Rename columns
df_uitkeringen.rename(columns={'Plaats': 'Gemeente','Uitkeringsontvangers, totaal |Uitkeringsontvangers, totaal': 'Uitkeringsontvangers totaal'}, inplace=True)

# Now we filter the dataframe to only contain the data we need
# Specify the columns to keep
columns_to_keep = ['Perioden', 'Gemeente', 'Uitkeringsontvangers totaal',
                   'Uitkeringsontvangers per soort uitkering|Werkloosheid',
                   'Uitkeringsontvangers per soort uitkering|Arbeidsongeschiktheid|Wajong-uitkering']

# Drop columns that are not in the specified list
df_uitkeringen = df_uitkeringen.drop(df_uitkeringen.columns.difference(columns_to_keep), axis=1)

del columns_to_keep

# Display the modified DataFrame
df_uitkeringen

,Perioden,Gemeente,Uitkeringsontvangers totaal,Uitkeringsontvangers per soort uitkering|Werkloosheid,Uitkeringsontvangers per soort uitkering|Arbeidsongeschiktheid|Wajong-uitkering
0,2019 oktober,Weert,14930,720,530
1,2019 november,Weert,14950,720,530
2,2019 december,Weert,14990,760,530
3,2020 januari,Weert,14960,720,530
4,2020 februari,Weert,14990,730,540
...,...,...,...,...,...
391,2022 mei*,Stein,8400,200,250
392,2022 juni*,Stein,8400,200,250
393,2022 juli*,Stein,8410,210,250
394,2022 augustus*,Stein,8410,200,250


In [51]:
# df_welzijn is het enige dataframe waaraan niks gedaan hoeft te worden. Dit is omdat het al op de juiste manier is geprepareerd.
df_welzijn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Gemeente                  353 non-null    object
 1   SES-WOA totaalscore 2019  352 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


In [52]:
# Converting DeelnemerID to INT to merge properly
df_deelnemeners['DeelnemerID'] = df_deelnemeners['DeelnemerID'].astype(pd.Int64Dtype())

# Merge df_deelnemers & df_fitheids_scan on 'DeelnemerID' for a complete DataFrame like the original excel file
df_healthy_life = pd.merge(df_fitheids_scan, df_deelnemeners, on='DeelnemerID')

# Delete df_deelnemers & df_fitheids_scan for minimalizing the ram usage
del df_fitheids_scan, df_deelnemeners

df_healthy_life.info()

df_healthy_life

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DeelnemerID  3190 non-null   Int64  
 1   Datum        3190 non-null   object 
 2   Lengte       3188 non-null   float64
 3   Gewicht      3171 non-null   float64
 4   6 MWT        3113 non-null   float64
 5   TUG          3123 non-null   float64
 6   BMI          3189 non-null   float64
 7   Conditie     3190 non-null   float64
 8   Lenigheid    2999 non-null   float64
 9   Knijpkracht  3190 non-null   float64
 10  #meting      3190 non-null   int64  
 11  Geslacht     3190 non-null   object 
 12  age          3190 non-null   int64  
 13  Gemeente     3190 non-null   object 
dtypes: Int64(1), float64(8), int64(2), object(3)
memory usage: 352.2+ KB


,DeelnemerID,Datum,Lengte,Gewicht,6 MWT,TUG,BMI,Conditie,Lenigheid,Knijpkracht,#meting,Geslacht,age,Gemeente
0,1,2018-03-30,159.0,97.6,500.0,6.36,38.606068,0.820291,31.50,15.0,1,V,51,Sittard-Geleen
1,1,2018-06-18,159.0,95.3,448.0,9.30,37.700000,0.695091,0.00,45.8,2,V,51,Sittard-Geleen
2,2,2018-03-30,175.0,89.1,596.0,5.82,29.093878,0.806495,27.00,34.0,1,V,45,Sittard-Geleen
3,2,2019-01-25,170.0,86.0,556.0,5.50,29.757785,0.779192,29.25,29.0,3,V,45,Sittard-Geleen
4,3,2018-07-02,151.5,94.8,512.0,6.84,41.303140,1.054387,19.00,20.3,2,V,68,Sittard-Geleen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,2412,2023-03-07,166.5,105.5,472.0,5.00,38.056074,0.770499,24.50,29.0,1,V,51,Kerkrade
3186,2408,2023-03-07,160.0,114.0,473.0,7.00,44.531250,0.898147,12.00,27.5,1,V,58,Kerkrade
3187,2223,2023-03-07,157.5,87.5,482.0,5.00,35.273369,0.704833,23.00,27.3,1,V,35,Kerkrade
3188,2407,2023-03-07,159.5,113.0,481.0,6.00,44.417803,0.984385,15.50,30.5,1,V,64,Kerkrade


In [53]:
# currently the memory usage is 2.5+ GB after the garbage collection. This is easily runnable on any modern computer.
gc.collect()

0

In [54]:
# Merging all the dataframes into one big dataframe named df_complete

